In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using YAXArrays
using Zarr
using Dates

  Activating project at `~/spatial-temporal-stuff/spatio_temporal_data-rainforest_development`
[ Info: Precompiling YAXArrays [c21b50f5-aa40-41ea-b809-c0f5e47bfa5c]


In [10]:
include("./rainforest_lib.jl")
using .RainforestLib

In [11]:
lc = RainforestLib.get_lcc_datacube()


┌ Warning: connection_limit no longer supported as a keyword argument; use `HTTP.set_default_connection_limit!(25)` before any requests are made or construct a shared pool via `POOL = HTTP.Pool(25)` and pass to each request like `pool=POOL` instead.
└ @ HTTP.Connections ~/.julia/packages/HTTP/uAaAw/src/Connections.jl:434
┌ Warning: connection_limit no longer supported as a keyword argument; use `HTTP.set_default_connection_limit!(25)` before any requests are made or construct a shared pool via `POOL = HTTP.Pool(25)` and pass to each request like `pool=POOL` instead.
└ @ HTTP.Connections ~/.julia/packages/HTTP/uAaAw/src/Connections.jl:434
┌ Warning: connection_limit no longer supported as a keyword argument; use `HTTP.set_default_connection_limit!(25)` before any requests are made or construct a shared pool via `POOL = HTTP.Pool(25)` and pass to each request like `pool=POOL` instead.
└ @ HTTP.Connections ~/.julia/packages/HTTP/uAaAw/src/Connections.jl:434
┌ Warning: connection_limit no 

YAXArray with the following dimensions
lon                 Axis with 129600 Elements from -179.9986111111111 to 179.99861111111113
lat                 Axis with 64800 Elements from 89.99861111111113 to -89.99861111111112
time                Axis with 11 Elements from 2010-01-01T00:00:00 to 2020-01-01T00:00:00
name: lccs_class
Total size: 86.03 GB


In [12]:
filtered_dc = RainforestLib.rough_spatial_filter(lc)

YAXArray with the following dimensions
lon                 Axis with 21600 Elements from 30.001388888888904 to 89.99861111111113
lat                 Axis with 16200 Elements from 14.99861111111112 to -29.99861111111111
time                Axis with 2 Elements from 2010-01-01T00:00:00 to 2011-01-01T00:00:00
name: lccs_class
Total size: 667.42 MB
